
<h3><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

# Embeddings

В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
# !gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
# !unzip archive.zip

Archive:  archive.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


### Импорты

In [1]:
import math
import random
import string
import re

import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [2]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [4]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [5]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [4]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [44]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ joy_inc well done you will be making progress in no time


In [45]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

well done will making progress time


Загрузим предобученную модель эмбеддингов. 

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [5]:
word2vec = api.load("word2vec-google-news-300")

In [47]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 

In [48]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [6]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        tokens = self.tokenizer.tokenize(text)
        filtered_tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
    
        return filtered_tokens


    def get_embeddings_(self, tokens):
        embeddings = [(word2vec.get_vector(w)-self.mean)/self.std for w in tokens if w in word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [8]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

<!-- Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть. -->

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [89]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))


1280


Сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

In [90]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(np.array(examples["features"]))

In [91]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [92]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1115', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [9]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [10]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)
        
        model.zero_grad()  
        pred = model(features)        
        
        loss = criterion(pred,targets)
        loss.backward() 
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            pred = model(features)
            loss = criterion(pred,targets)
            acc = sum(torch.argmax(pred,axis = 1) == targets)/len(targets)

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [95]:
import torch.nn as nn
from torch.optim import Adam


vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-4
num_epochs = 3

model = nn.Sequential(
            nn.Linear(vector_size,500),
            nn.Sigmoid(),
            nn.Linear(500,200),
            nn.Sigmoid(),
            nn.Linear(200,100),
            nn.Sigmoid(),
            nn.Linear(100,num_classes),
            nn.Sigmoid()
        )
model = model.to(torch.device(device))
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr = lr)

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [96]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5757437865734101, 'Test Acc': 0.720390625}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5728771069049835, 'Test Acc': 0.721546875}


Epoch 3. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5711968140602112, 'Test Acc': 0.72237890625}


In [97]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5702286261720018, 'Test Acc': 0.7237201727236422}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

### Эмбеддинги через взвешенную сумму 

In [13]:
def get_tokens(text):
    pattern='[A-z]+|@[A-z]+'
    tokenizer = nltk.RegexpTokenizer(pattern=pattern)
    tokens = tokenizer.tokenize(text)
    filtered_tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
    return filtered_tokens

In [99]:
class DatasetUnknwnEmbs(TwitterDataset):
    
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, window_size: int):
        super().__init__(data,feature_column,target_column,word2vec)
         # не отделять @ от никнейма, чтобы стирать никнеймы и не учить эмбеддинги 
        pattern='[A-z]+|@[A-z]+'
        self.tokenizer = nltk.RegexpTokenizer(pattern=pattern )
        self.window_size = window_size

    
    def get_embeddings_(self, tokens):
        unknown_idxs  = [i for i,w in enumerate(tokens) if w not in self.word2vec]     
        embeddings =[(word2vec.get_vector(w)-self.mean)/self.std \
                                    if w in self.word2vec \
                                    else np.zeros(self.word2vec.vector_size)\
                                    for w in tokens ]
        
        embeddings = np.array(embeddings)

        n = self.window_size # вызвешенно учитываю по n слов из контекста влево и вправо
        for i in unknown_idxs:
#             границы окна контекста
            lb=  i-n if i-n>0 else 0
            rb= i+n+1 if i+n<len(embeddings) else len(embeddings)
            
#             веса соседних слов
            l_arr = np.arange(1,i-lb+1)
            r_arr = np.arange(1,rb-i)
            
            w = [np.concatenate([1/l_arr[::-1],[0],1/r_arr])]*self.word2vec.vector_size
            w = np.stack(w,axis=1)
#             подсчет вызвешенной суммы
            embeddings[i] = np.sum(embeddings[lb:rb]*w,axis =0)


        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings
    
     

In [100]:
len_list = data['text'].map(get_tokens).map(len)
len_list.mean()

6.834416875

In [101]:
# возьмем размер окна контекста как средний размер предложения 

win_size = 7
dev_ue = DatasetUnknwnEmbs(dev_data, "text", "emotion", word2vec, win_size)

In [102]:
train_size = math.ceil(len(dev_ue) * 0.8)
num_workers = 2
train_ue, valid_ue = random_split(dev_ue, [train_size, len(dev_ue) - train_size])

train_loader_ue = DataLoader(train_ue, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader_ue = DataLoader(valid_ue, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [103]:
vector_size = dev_ue.word2vec.vector_size
num_classes = 2
lr = 1e-4
num_epochs = 3

model_ue = nn.Sequential(
            nn.Linear(vector_size,500),
            nn.Sigmoid(),
            nn.Linear(500,200),
            nn.Sigmoid(),
            nn.Linear(200,100),
            nn.Sigmoid(),
            nn.Linear(100,num_classes),
            nn.Sigmoid()
        )
model_ue = model_ue.to(torch.device(device))
criterion = nn.CrossEntropyLoss()
optimizer_ue = Adam(model_ue.parameters(),lr = lr)

In [104]:
best_metric = np.inf
for e in range(num_epochs):
    training(model_ue, optimizer_ue, criterion, train_loader_ue, e, device)
    log = testing(model_ue, criterion, valid_loader_ue, device)
    if log["Test Loss"] < best_metric:
        torch.save(model_ue.state_dict(), "model_ue.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 3. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

In [105]:
test_loader_ue = DataLoader(
    DatasetUnknwnEmbs(test_data, "text", "emotion", word2vec,win_size), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model_ue.load_state_dict(torch.load("model_ue.pt", map_location=device))

print(testing(model_ue, criterion, test_loader_ue, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5710094199774746, 'Test Acc': 0.7229776108226837}


### TF-IDF эмбеддинги


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sparsesvd import sparsesvd

def get_tfidf_embeddings(data,tokenizer,v_size):
    tfidf = TfidfVectorizer(tokenizer= tokenizer)
    res = tfidf.fit_transform(data)
    u, s, vt = sparsesvd(res.tocsc(),v_size)
    k,v = tfidf.get_feature_names_out() , vt.T
    v = (v-v.mean(axis=0))/v.std(axis=0)
    return dict(zip(k,v))



In [14]:
tfidf_embeddings_dict = get_tfidf_embeddings(data['text'],get_tokens,word2vec.vector_size)

In [15]:
class DatasetTFIDFEmbs(TwitterDataset):
    
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, tfidf_embeddings_dict: dict):
        super().__init__(data,feature_column,target_column,word2vec)
#          # не отделять @ от никнейма, чтобы стирать никнеймы 
        pattern='[A-z]+|@[A-z]+'
        self.tokenizer = nltk.RegexpTokenizer(pattern=pattern )
        self.tfidf_dict = tfidf_embeddings_dict

    
    def get_embeddings_(self, tokens):
        unknown_idxs  = [i for i,w in enumerate(tokens) if w not in self.word2vec]     
        w2v_embeddings =[(word2vec.get_vector(w)-self.mean)/self.std \
                                    if w in self.word2vec \
                                    else np.zeros(self.word2vec.vector_size)\
                                    for w in tokens ]
        w2v_embeddings = np.array(w2v_embeddings)
        tfidf_embeddings = [self.tfidf_dict[w.lower()] for w in tokens]
        embeddings = w2v_embeddings + tfidf_embeddings

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings
    
     

In [16]:
dev_tfidf = DatasetTFIDFEmbs(dev_data, "text", "emotion", word2vec,tfidf_embeddings_dict)

In [17]:
train_size = math.ceil(len(dev_tfidf) * 0.8)
num_workers = 2
train_tfidf, valid_tfidf = random_split(dev_tfidf, [train_size, len(dev_tfidf) - train_size])

train_loader_tfidf = DataLoader(train_tfidf, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader_tfidf = DataLoader(valid_tfidf, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [18]:
import torch.nn as nn
from torch.optim import Adam


vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-4
num_epochs = 3

model_tfidf = nn.Sequential(
            nn.Linear(vector_size,500),
            nn.Sigmoid(),
            nn.Linear(500,200),
            nn.Sigmoid(),
            nn.Linear(200,100),
            nn.Sigmoid(),
            nn.Linear(100,num_classes),
            nn.Sigmoid()
        )
model_tfidf = model_tfidf.to(torch.device(device))
criterion = nn.CrossEntropyLoss()
optimizer_tfidf = Adam(model_tfidf.parameters(),lr = lr)

In [19]:
best_metric = np.inf
for e in range(num_epochs):
    training(model_tfidf, optimizer_tfidf, criterion, train_loader_tfidf, e, device)
    log = testing(model_tfidf, criterion, valid_loader_tfidf, device)
    if log["Test Loss"] < best_metric:
        torch.save(model_tfidf.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 3. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

In [20]:
test_loader_tfidf = DataLoader(
    DatasetTFIDFEmbs(test_data, "text", "emotion", word2vec,tfidf_embeddings_dict), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model_tfidf.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model_tfidf, criterion, test_loader_tfidf, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5690574756445596, 'Test Acc': 0.7264283396565495}


## Вывод

Итоговые результаты:


| embeddings |Test Loss |Test Acc |
|------|------|------|
| average  |0.5702| 0.7237|
| weighted sum |0.5710| 0.7229|
| tfidf |0.5690| 0.7264|

Лучшим себя показал метод с tf-idf эмбеддингами. Полагаю, это связанно с тем, что в данном методе при создании эмбеддингов использовалась матрица tf-idf рассчитанная на всем корпусе текстов. При этом обновлялись эмбеддинги не только слов, которых нет в словаре w2v модели, но и всех остальных. Значи такие эмбеддинги заключают в себе большее количество информации и, соответсвенно, дают больше возможностей для предсказания.

Эмбеддинги, основанные на сумме взвешенных весов показывают результат чуть худший, чем исходные. Это может быть связано с тем, что формируя эмбеддинги неизвестных слов как сумму взвешенных соседей учитывается упоминание этого слова только в одном предложении - в одном контексте. Такой эмбеддинг получается не репрезентативным и, видимо, даже мешает классификации